The data came from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE171524 --> GSE171524_RAW.tar file

## Integration

Only necessary if you are runing an analysis with multiple samples. The experiment sampled 19 COVID-19 patients and 7 control patients. There are a total of 26 samples in this analysis, thus, integration is necessary.

We are essentially gonna now do the same thing as we did for (1) loading the data, (2) doublet removal, (3) preprocessing, and (4) Clustering.
However, we will do this for all samples, to make this more efficient, we made this into a function and have the function iterate through each sample file.

In [1]:
import gc  # Import garbage collector
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import scanpy as sc
import scvi
import pandas as pd
import numpy as np
import os

def pp(csv_path):
    adata = sc.read_csv(csv_path).T
    sc.pp.filter_genes(adata, min_cells=10)
    sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True, flavor='seurat_v3')
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df['prediction'] = solo.predict(soft=False)
    df.index = df.index.map(lambda x: x[:-2])
    df['dif'] = df.doublet - df.singlet
    doublets = df[(df.prediction == 'doublet') & (df.dif > 0.25)]
    
    adata = sc.read_csv(csv_path).T
    adata.obs['Sample'] = csv_path.split('_')[2]
    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)
    adata = adata[~adata.obs.doublet]
    
    sc.pp.filter_cells(adata, min_genes=200)
    adata.var['mt'] = adata.var_names.str.startswith('mt-')
    
    ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"
    ribo_genes = pd.read_table(ribo_url, skiprows=2, header=None)
    adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
    
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
    
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    adata = adata[adata.obs.n_genes_by_counts < upper_lim]
    adata = adata[adata.obs.pct_counts_mt < 20]
    adata = adata[adata.obs.pct_counts_ribo < 2]
    
    return adata

def batch_process(input_dir, output_dir, batch_size=2):  # Reduce batch size to 2
    files = os.listdir(input_dir)
    batches = [files[i:i + batch_size] for i in range(0, len(files), batch_size)]
    
    for i, batch in enumerate(batches):
        out = []
        for file in batch:
            file_path = os.path.join(input_dir, file)
            out.append(pp(file_path))
        
        batch_adata = sc.concat(out)
        batch_adata.write(os.path.join(output_dir, f'batch_{i}.h5ad'))
        print(f'Processed batch {i} and saved to disk.')
        
        del out, batch_adata  # Clear variables to free memory
        gc.collect()  # Force garbage collection

c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set your directories and batch size
input_dir = 'GSE171524_RAW/'
output_dir = 'D:/processed_batches/'
os.makedirs(output_dir, exist_ok=True)

# Run batch processing with reduced batch size
batch_process(input_dir, output_dir, batch_size=2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 400/400: 100%|██████████| 400/400 [15:05<00:00,  2.23s/it, v_num=1, train_loss_step=331, train_loss_epoch=323]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [15:06<00:00,  2.27s/it, v_num=1, train_loss_step=331, train_loss_epoch=323]
INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


...

Epoch 400/400: 100%|██████████| 400/400 [17:18<00:00,  2.60s/it, v_num=1, train_loss_step=319, train_loss_epoch=341]
INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 172/400:  43%|████▎     | 172/400 [01:11<01:34,  2.41it/s, v_num=1, train_loss_step=0.223, train_loss_epoch=0.314]
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.293. Signaling Trainer to stop.


c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\torch\utils\_contextlib.py:116: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabiltiies. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)
c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\torch\utils\_contextlib.py:116: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabiltiies. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)
c:\Users\agarc\anaconda3\envs\bioenv\Lib\site-packages\scanpy\preprocessing\_simple.py:166: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"

Processed batch 12 and saved to disk.


In [3]:
# Later, you can concatenate the batches
batches = [sc.read(os.path.join(output_dir, f)) for f in os.listdir(output_dir) if f.endswith('.h5ad')]
adata = sc.concat(batches)
sc.pp.filter_genes(adata, min_cells=10)

MemoryError: Unable to allocate 1.24 GiB for an array with shape (9647, 34546) and data type float32

1. Sequential Processing with Incremental Merging: Instead of loading all .h5ad files at once, process them one by one or in smaller groups, then combine the results incrementally.

In [5]:
output_dir = 'D:/processed_batches/'
merged_adata = None

for file in os.listdir(output_dir):
    if file.endswith('.h5ad'):
        print(f"Processing {file}...")
        adata = sc.read(os.path.join(output_dir, file))
        
        if merged_adata is None:
            merged_adata = adata
        else:
            merged_adata = sc.concat([merged_adata, adata])
        
        del adata
        gc.collect()

# Save the final merged AnnData object
merged_adata.write(os.path.join(output_dir, 'merged_data.h5ad'))


Processing batch_0.h5ad...
Processing batch_1.h5ad...
Processing batch_2.h5ad...
Processing batch_3.h5ad...
Processing batch_4.h5ad...
Processing batch_5.h5ad...


MemoryError: Unable to allocate 7.08 GiB for an array with shape (55051, 34546) and data type float32

In [ ]:
sc.pp.filter_genes(merged_adata, min_cells=10)